In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read data from file 'Tenant_Directory.xls' 
data = pd.read_excel('Tenant_Directory.xls')

# Preview the first 5 lines of the loaded data
data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,102.0,"Jardine, Angus",NaN,NaN,NaN,4.165882e+09,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,103.0,"McKay, Alistair",NaN,NaN,NaN,NaN,NaN,4165365016
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,104.0,"Kiflezgi, Yared Tesfazgi",NaN,NaN,NaN,4.165310e+09,NaN,4168805494


In [2]:
#Remove the useless columns
data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 6'], axis=1)
data.columns = ['SuitName','Name','PhoneNumber','AltPhoneNumber']

#Remove all rows which have Null SuitName
data = data.dropna(subset=['SuitName'])

#Convert the SuitName from float to int
data['SuitName'] = data['SuitName'].astype(int)

#There are 11 floors, 21 apartments per floor, from 1-21
#The apartments which are not in the file are needed to insert to the file. 
for i in range(1,12):
    floor = list(range(i*100+1,i*100+22))
    data_floor = data[(data['SuitName'] >= i*100) & (data['SuitName'] <= (i+1)*100)]
    temp = []
    for x in floor:
        for y in data_floor['SuitName']:
            if x == y:
                temp.append(x)
    
    for z in list(set(floor) - set(temp)):
        row = pd.DataFrame({'SuitName':[z]})
        data = data.append(row)

#Reorder the columns        
data = data[['SuitName','Name','PhoneNumber','AltPhoneNumber']]

#Sort the data based on the SuitName
data = data.sort_values(by=['SuitName'], ascending=True)

#Fill other Null values with 0
data.fillna(0, inplace=True)

#Reindex the whole table with SuitID, starting from 1
data = data.reset_index(drop=True)
data.index += 1

#Create a SuitID as the index and obtain the required format of mesh system
data['SuitID'] = data.index

#Create a new SuitCode column
data['SuitCode'] = data['SuitName']

#The mesh system requires SuitCode need to have 4 digits and not duplicated 
for i in data['SuitCode']:
    if i <= 999:
        data = data.replace({'SuitCode':{i:str(i) + '0'}})

#Convert all SuitCode values to string
data['SuitCode'] = data['SuitCode'].astype(str)

#Check that there is any duplicated SuitCode
duplicateRows = data[data.duplicated(['SuitCode'], keep=False)]['SuitCode']
print(duplicateRows)

data.at[1, 'SuitCode']='2222'
data.at[2, 'SuitCode']='3333'
data.at[11, 'SuitCode']='4444'
data.at[12, 'SuitCode']='5555'

duplicateRows = data[data.duplicated(['SuitCode'], keep=False)]['SuitCode']
print(duplicateRows)

#New data frame with split value columns 
new = data['Name'].str.split(", ", n = 1, expand = True) 
  
#Making separate first name column from new data frame 
data['LastName']= new[0] 
  
#Making separate last name column from new data frame 
data['FirstName']= new[1] 
  
#Adding new columns for required format of mesh system
data['SuitName'] = data['SuitName'].astype(str)
data['SuitNumber'] = 'Main: ' + data['SuitName']
data = data[['SuitID', 'SuitName', 'SuitCode', 'PhoneNumber', 'AltPhoneNumber', 'FirstName', 'LastName', 'Name', 'SuitNumber']]

data.head(20)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


1      1010
2      1020
11     1110
12     1120
199    1010
209    1020
220    1110
230    1120
Name: SuitCode, dtype: object
Series([], Name: SuitCode, dtype: object)


,SuitID,SuitName,SuitCode,PhoneNumber,AltPhoneNumber,FirstName,LastName,Name,SuitNumber
1,1,101,2222,0.000000e+00,0,NaN,NaN,0,Main: 101
2,2,102,3333,4.165882e+09,0,Angus,Jardine,"Jardine, Angus",Main: 102
3,3,103,1030,0.000000e+00,4165365016,Alistair,McKay,"McKay, Alistair",Main: 103
4,4,104,1040,4.165310e+09,4168805494,Yared Tesfazgi,Kiflezgi,"Kiflezgi, Yared Tesfazgi",Main: 104
5,5,105,1050,4.168906e+09,0,Momtahan,Sina,"Sina, Momtahan",Main: 105
6,6,106,1060,4.162830e+09,4165360397,Dookie,Dolly,"Dolly, Dookie",Main: 106
7,7,107,1070,0.000000e+00,0,NaN,NaN,0,Main: 107
8,8,108,1080,0.000000e+00,0,NaN,NaN,0,Main: 108
9,9,109,1090,6.477043e+09,0,None,Junne Delos Santos,Junne Delos Santos,Main: 109
10,10,110,1100,0.000000e+00,5195675841,Bojan,Blagojevic,"Blagojevic, Bojan",Main: 110


In [3]:
#Export to the CSV file to import to the mesh system
data.to_csv('Data_for_mess_system.csv', encoding='utf-8', index=False)